## Importing libraries

In [1]:
# Importing libraries
import json
import time
from telnetlib import Telnet
import serial
import pandas as pd
import pickle

import warnings
warnings.filterwarnings('ignore')

## Initializing keys, PORT and telnet configuration

In [2]:
# Initializing keys, PORT and telnet configuration
ser = serial.Serial('COM3', 9600, timeout=1);
tn=Telnet('localhost',13854);

start=time.clock();

i=0;
tn.write(b'{"enableRawOutput": true, "format": "Json"}');

eSenseDict={'attention':0, 'meditation':0};
waveDict={'lowGamma':0, 'highGamma':0, 'highAlpha':0, 'delta':0, 'highBeta':0, 'lowAlpha':0, 'lowBeta':0, 'theta':0};
signalLevel=0;
signalValue = "";

open_data = []
close_data = []
fingopen= 1;
fingclose = 1;

def hzconvert(s):
    return s[-5:2]

## Loading Model

In [3]:

def predict_mpg(config):
    ##loading the model from the saved file
    pkl_filename = "model.pkl"
    with open(pkl_filename, 'rb') as f_in:
        model = pickle.load(f_in)
    
    if (config):
        df = pd.DataFrame(config)
        
    else:
        df = config
    
    y_pred = model.predict(df)
    y_pred_int = int(y_pred[0])
    return y_pred_int

## Main Algorithm

In [4]:
while i<100:
    blinkStrength=0;
    # read the input array
    line=tn.read_until(b'\r');
    
    
    if len(line) > 20:
        timediff=time.clock()-start;
        dict=json.loads((line));
        
        # check the mind headset values 
        if "poorSignalLevel" in dict:
            signalLevel=dict['poorSignalLevel'];
        if "blinkStrength" in dict:
            blinkStrength=dict['blinkStrength'];
        if "eegPower" in dict:
            waveDict=dict['eegPower'];
            eSenseDict=dict['eSense'];

        # get the waves value
        attentionStr=str(eSenseDict['attention']);
        meditationStr=str(eSenseDict['meditation']);
        lowGamma=hzconvert(str(waveDict['lowGamma']));
        highGamma=hzconvert(str(waveDict['highGamma']));
        highAlpha=hzconvert(str(waveDict['highAlpha']));
        delta=hzconvert(str(waveDict['delta']));
        highBeta=hzconvert(str(waveDict['highBeta']));
        lowAlpha=hzconvert(str(waveDict['lowAlpha']));
        lowBeta=hzconvert(str(waveDict['lowBeta']));
        theta=hzconvert(str(waveDict['theta']));
        
        # check the blink and time
        blinkStrengthStr=str(blinkStrength);
        seconds = time.time();        
     
        if(signalLevel==200):
            signalValue = "Poor";
        else:
            signalValue = "Strong";
        
        if(signalValue == "Strong"):
            
            config = {'lowGamma': [int(lowGamma)],
                      'highGamma': [int(highGamma)],
                      'highAlpha': [int(highAlpha)],
                      'delta': [0],
                      'highBeta': [int(highBeta)],
                      'lowAlpha': [int(lowAlpha)],
                      'lowBeta': [int(lowBeta)],
                      'theta': [int(theta)]}

            #  Check the values using model(0 - close ? 1 - Open) 
            result = predict_mpg(config)

            if (result == 0):
                close_data.append(result)
                if(len(close_data) > 3 ):
                    ser.write(str.encode('b'));
                    print("Close Fingers");
                    time.sleep(1)

            elif (result == 1):
                open_data.append(result)
                if(len(open_data) > 3 ):
                    ser.write(str.encode('a'));
                    print("Open Fingers");
                    time.sleep(1)
                
            else:
                open_data.clear()
                close_data.clear()
                print("Out of control");

        else:
            print("Poor Signal");

            
ser.close();
tn.close();

Poor Signal
Poor Signal
Poor Signal
Poor Signal
Poor Signal
Poor Signal
Poor Signal
Poor Signal
Poor Signal
Close Fingers
Close Fingers
Close Fingers
Close Fingers
Close Fingers
Close Fingers
Close Fingers
Close Fingers
Close Fingers
Close Fingers
Close Fingers
Open Fingers
Open Fingers
Open Fingers
Open Fingers
Open Fingers
Close Fingers
Close Fingers
Close Fingers
Open Fingers
Open Fingers
Close Fingers
Close Fingers
Close Fingers
Open Fingers
Open Fingers
Close Fingers
Close Fingers
Close Fingers
Open Fingers
Close Fingers
Open Fingers
Close Fingers
Open Fingers
Open Fingers
Open Fingers
Open Fingers
Open Fingers
Close Fingers
Open Fingers
Open Fingers
Close Fingers
Open Fingers
Open Fingers
Open Fingers
Close Fingers
Close Fingers
Open Fingers
Open Fingers
Close Fingers
Open Fingers
Close Fingers
Close Fingers
Open Fingers
Close Fingers
Open Fingers
Open Fingers
Open Fingers
Open Fingers
Open Fingers
Open Fingers
Open Fingers
Open Fingers
Open Fingers
Open Fingers
Close Fingers
Clo

ValueError: invalid literal for int() with base 10: ''